# results vary

### HistGradientBoostingRegressor is better for datasets smaller than 10,000

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

from sklearn import ensemble
from sklearn.inspection import permutation_importance

In [2]:
df = pd.read_csv('hotData.csv')


In [3]:
df.head()

,goals,xg,crosses,boxtouches,passes,progpasses,takeons,progruns,tackles,interceptions,...,64,65,66,67,68,69,70,71,72,73
0,0.055385,-0.119231,-0.020769,0.236154,-0.781538,-0.649231,-0.407692,-0.568462,-0.578462,-0.444615,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.243077,0.063846,-0.090769,-0.287692,-0.704615,-0.441538,-0.098462,-0.383077,-0.722308,-0.606154,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.420000,-0.230000,-0.150769,-0.435385,-0.687692,-0.408462,-0.256923,-0.586154,-0.629231,-0.560000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.420000,-0.230000,-0.150769,-0.435385,-0.687692,-0.408462,-0.256923,-0.586154,-0.629231,-0.560000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.070769,-0.368462,-0.493846,-0.247692,-0.681538,-0.536923,-0.210000,-0.427692,-0.539231,-0.622308,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# df.columns

In [24]:
y = df['results']
X = df.drop(columns=['results'])

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.10, random_state=2)

In [25]:
params = {}

In [26]:
#     "n_estimators": 3990,
#     "max_depth": 400,
#     "min_samples_split":4,
#     "learning_rate": 0.0159,
#     "loss": "squared_error",
#     "max_features": 16,
#     "min_weight_fraction_leaf": 0.11,
# #     "subsample": 0.8
#     "min_impurity_decrease": 5,

In [27]:
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)

print('GRADIENT BOOSTING REGRESSION')

test_pred = reg.predict(X_test)

print('RMSE: '+str(np.sqrt(mean_squared_error(y_test, test_pred))))
print('R Squared: '+str(r2_score(y_test, test_pred)))
print('MSE TEST: '+str(mean_squared_error(y_test, test_pred)))
print('MAE TEST: '+str(mean_absolute_error(y_test, test_pred)))
print('MAPE: '+str(mean_absolute_percentage_error(y_test, test_pred)))


GRADIENT BOOSTING REGRESSION
RMSE: 7.186122274134917
R Squared: 0.3516624793547022
MSE TEST: 51.64035333881799
MAE TEST: 5.470810102436329
MAPE: 0.7539853922939889


In [8]:
# varies
# MAPE: 0.7479741668404841
# MAPE: 0.7343087901621291

# MAPE: 0.7539853922939889


In [9]:
# mape = 1.0765
# mape = 1.0647916900630887 max_feat = 16 
# mape 1.059828673697475 / estimators 5000
# mape 1.0597337473020356 / estimators 4000
# mape 1.0597326684323638 / estimators 4990
# mape 1.0432022275261308 / min fraction leaf .12
#mape 1.031433617032885 / '' ''     ''   '' .11
# ** Results vary -mape .9879763523310932 / "subsample": 0.8
# mpae  1.0241404892305128 / min_impurity_decrease 10
# mape 1.0097789777918718 / min decrease 5

In [10]:
data = {'y_test': y_test, 'y_pred': test_pred}

In [11]:
new_df = pd.DataFrame(data)

In [12]:
diff = (new_df['y_test'] - new_df['y_pred']).abs()

In [13]:
new_df['diff'] = diff
new_df = new_df.sort_values(by='diff', ascending=True)
new_df['y_pred'] = round(new_df['y_pred'], 1)
new_df['rounded'] = round(new_df['y_pred'])
new_df['new_diff'] = (new_df['y_test'] - new_df['rounded']).abs()

In [14]:
new_df

,y_test,y_pred,diff,rounded,new_diff
106,9,9.0,0.018952,9.0,0.0
141,14,14.0,0.044520,14.0,0.0
11,12,12.2,0.162344,12.0,0.0
144,4,4.3,0.335146,4.0,0.0
66,17,17.5,0.515431,18.0,1.0
130,10,11.2,1.201389,11.0,1.0
151,22,20.6,1.420234,21.0,1.0
29,23,21.1,1.866983,21.0,2.0
101,1,4.1,3.141860,4.0,3.0
77,18,14.5,3.499495,14.0,4.0


In [15]:
new_df['diff'].mean()

5.393315939715033

In [16]:
new_df['diff'].median()

4.422947246375236

In [17]:
new_df['new_diff'].mean()


5.379310344827586

In [18]:
new_df['new_diff'].median()

4.0

In [19]:
# from sklearn.linear_model import Ridge
# from sklearn.feature_selection import SequentialFeatureSelector
# from sklearn.model_selection import TimeSeriesSplit

# rr = Ridge(alpha=1)

# split = TimeSeriesSplit(n_split=3)

# sfs = SequentialFeatureSelector(rr, features_to_select=17, 
#                                 direction="forward", cv=split, 
#                                 n_jobs=4)


In [20]:
df.shape

(282, 91)